In [2]:
import string
import re
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
cleaned_data = pd.read_csv('../../cleaned_data.csv')

In [4]:
cleaned_data.tail(5)

,title,content
53782,ኦዚ ደርሶ መልስ ኤፍሬም ማዴቦ መልሱን እነሆ አንዱአለም ተፈራ,አንዱአለም ተፈራ አርብ ግንቦት ቀን አመተ ምህረት ኦዚ ደርሶ መልስ ...
53783,ቴዲ አፍሮ በኢህአዴግ ለምን ይጠላል ወይም እንደ ትልቅ አደጋ ይታያል,ከጌታቸው ስሜ ቴዲ አፍሮ የሚነያሳቸው የኢትዮጵያዊነት ወይም የተለያየ ማህ...
53784,ትራምፕ ሙስሊሞችን ከማገድ ወደ ሙስሊም አገር መንጎድ በሳዲቅ አህመድ ...,በኤፍቢአይ ምርመራ የፖለቲካ አጣብቂኝ ዉስጥ የገቡት ዶናልድ ትራምፕ ከ የ...
53785,ከቴዲ አፍሮ ጋር ያደረገው ቃለምልልስ በኢቢሲ የታገደበት ጋዜጠኛ ስራውን ...,ዘሀበሻ ቴዲ አፍሮ መኖሪያ ቤት ድረስ ሄዶ ቃለምልልስ አድርጎ እንዳይተላለ...
53786,ራእይአቹ ምንድን ነው ከተማ ዋቅጅራ,ራእይ የሚያይ ቅን ሰው ራእይ የሚፈታ ቅዱስ አባት ያስፈልጋል። የኢትዮጵያ...


In [13]:
class ExtractiveModel:

    am_sent_endings = r'\?|\!|\።|\፡፡'
    am_punctuation = '፠፡።፣፤፥፦፧፨“”‘’…‹‹››·•'
    am_numbers = '፩፪፫፬፭፮፯፰፱፲፳፴፵፶፷፸፹፺፻፼'
    am_random = '�©\uf0c4\uf0d8\uf0a7\uf066\uf0d8'
    stop_words = open('stopwords.txt', encoding='utf-8').read().split()

    def __init__(self, text):
        self.raw_text = text
        self.clean_text = None
        self.sentences = None
        self.words = None

        self.process_text()

    def process_text(self):
        self.clean_text = self.clean_minimized(self.raw_text)
        self.sentences = self.extract_sentences(self.clean_text)
        self.sentences = self.remove_duplicate_sentence(self.sentences)
        self.words = self.extract_words(self.clean_text)

    def extract_sentences(self, text=None):
        if text is None:
            text = self.raw_text
        sentences = re.split(self.am_sent_endings, text)
        return sentences

    def extract_words(self, text):
        return text.split()

    def clean_minimized(self, text):
        words = text.split()
        to_clean = string.punctuation + self.am_numbers + self.am_random + string.ascii_letters + string.digits + self.am_punctuation
        to_clean = re.sub(self.am_sent_endings, '', to_clean)
        table = str.maketrans('', '', to_clean)
        stripped = [w.translate(table) for w in words]
        clean_txt = list(filter(None, stripped))
        return ' '.join(clean_txt)

    def remove_duplicate_sentence(self, sentences):
        duplicates = []
        cleaned = []
        for s in sentences:
            if s in cleaned:
                if s in duplicates:
                    continue
                else:
                    duplicates.append(s)
            else:
                cleaned.append(s)
        return cleaned

In [14]:
def _create_dictionary_table(words) -> dict:
    frequency_table = dict()

    for wd in words:
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [15]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount_without_stop_words = 0
        words = sentence.split()
        for word in words:
            if word in frequency_table:
                sentence_wordcount_without_stop_words += frequency_table[word]

        sentence_weight[sentence] = sentence_wordcount_without_stop_words

    return sentence_weight


In [16]:
def _calculate_average_score(sentence_weight) -> int:
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    average_score = (sum_values / len(sentence_weight))

    return average_score

In [17]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        # print(sentence_weight)
        if sentence in sentence_weight and sentence_weight[sentence] >= threshold//2:
            article_summary += sentence + '።'
            sentence_counter += 1

    article_summary = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', article_summary, flags=re.I)

    return article_summary


In [18]:
def _get_summary(tparser, threshold_parameter=1.5):
    sentences = tparser.sentences

    frequency_table = _create_dictionary_table(tparser.words)

    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    threshold = _calculate_average_score(sentence_scores)

    return _get_article_summary(sentences, sentence_scores, threshold_parameter * threshold)



In [19]:
!pip install etnltk

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import nltk
from etnltk.tokenize.am import word_tokenize

In [21]:
from difflib import SequenceMatcher

def longest_common_subsequence(a, b):
    matcher = SequenceMatcher(None, a, b)
    match = matcher.find_longest_match(0, len(a), 0, len(b))
    return a[match.a: match.a + match.size]


In [29]:
def rouge_n(reference, candidate, n=1):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    
    reference_ngrams = set(nltk.ngrams(reference_tokens, n))
    candidate_ngrams = set(nltk.ngrams(candidate_tokens, n))
    
    precision = len(reference_ngrams.intersection(candidate_ngrams))
    recall = len(reference_ngrams) + len(candidate_ngrams) - precision
    
    if recall == 0:
        rouge_n_score = 0.0
    else:
        rouge_n_score = precision / recall
    
    return rouge_n_score

In [30]:
def rouge_l(reference, candidate):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    
    reference_ngrams_all = list(nltk.ngrams(reference_tokens, 1))
    candidate_ngrams_all = list(nltk.ngrams(candidate_tokens, 1))
    
    reference_lcs = longest_common_subsequence(reference_ngrams_all, candidate_ngrams_all)
    
    rouge_l_score = len(reference_lcs) / max(1, len(reference_ngrams_all))
    
    return rouge_l_score


In [31]:
def calculate_rouge_scores(reference, candidate):
    rouge_n_1 = rouge_n(reference, candidate, n=1)
    rouge_n_2 = rouge_n(reference, candidate, n=2)
    rouge_l_score = rouge_l(reference, candidate)
    
    return rouge_n_1, rouge_n_2, rouge_l_score

In [32]:
def rouge_score(reference_summary, generated_summary):
    rouge_n_1, rouge_n_2, rouge_l_score = calculate_rouge_scores(reference_summary, generated_summary)
    
    print(f"ROUGE-1: {rouge_n_1:.4f}")
    print(f"ROUGE-2: {rouge_n_2:.4f}")
    print(f"ROUGE-L: {rouge_l_score:.4f}")


In [33]:
X = cleaned_data['content']
y = cleaned_data['title']    

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
content = X_test.iloc[-20]
title = y_test.iloc[-20]
tparser = ExtractiveModel(content)
article_summary = _get_summary(tparser)
rouge_score(title, article_summary)
print("\n\n")
print("Text")
print("\n")
print(content)
print("===================")
print("Summarized")
print("\n")
print(article_summary)

ROUGE-1: 0.0625
ROUGE-2: 0.0323
ROUGE-L: 0.4000



Text


ባለፉት ሶስት አመታት በአደንዛዥ እፅ ሱሰኝነት በቤተሰብ ችግርና በሌሎች የገንዘብ ኪሳራ እክሎች ስትባክን የቆየችው የ አመቷ ሊንድሴይ ሎሀን በፊልሙ ላይ ለመስራት ጉጉት እንዳደረባት ሮይተርስ አውስቷል። ሊዝ ኤንድ ዲክ በሰኔ ወር የፊልሙ ቀረፃ እንደሚጀመር ሲታወቅ ሊንድሴይ ሎሀን በተሳካ ትወና የተዳከመ ዝና እና ገቢዋን በማስመለስ ልታገግምበት እንደምትችል ተገምቷል።ለመጨረሻ ጊዜ ፊልም ከሰራች ሶስት አመት ያለፋት ሎሀን ለትወናው በመመረጧ ታላቅ ክብር እንደተሰማት ብትገልፅም አንዳንድ ታዋቂ ሰዎች ለኤልዛቤት ቴይለር ስብእና የማትመጥን ናት በሚል ተቃውሞ እያሰሙ ነው። በሙያው በቆየችባቸው አስር አመታት ከ በላይ ፊልሞች የተወነችው ሊንድሴይ ሎሀን በአለም ዙርያ  ሚሊዮን ዶላር ገቢ ያስገኘች ስትሆን በአንድ ፊልም  ሚሊዮን ዶላር አማካይ ገቢ እንዳላት የቦክስኦፊስ ባይ መረጃ ያመለክታል። 
Summarized


ባለፉት ሶስት አመታት በአደንዛዥ እፅ ሱሰኝነት በቤተሰብ ችግርና በሌሎች የገንዘብ ኪሳራ እክሎች ስትባክን የቆየችው የ አመቷ ሊንድሴይ ሎሀን በአለም ዙርያ ሚሊዮን ዶላር አማካይ ገቢ እንዳላት የቦክስኦፊስ ባይ መረጃ ያመለክታል።


In [55]:
content = '''
  የዓለም የምጣኔ ሃብት ጉባኤ በዳቮስ፣ ስዊትዘርላንድ ባለፈው ሳምንት ተካሂዷል። ጎን ለጎን የዓለም ጤና የሚያሳስባቸው ትልልቅ መሪዎች በር ዘግተው ለቀናት መክረዋል። እንደ ኮቪድ ያለ አዲስ ወረርሽኝ ከመምጣቱ በፊት ምን ይደረግ? እንዴት እንዘጋጅ በሚለው አጀንዳ ነው የተወያዩት።

ለጊዜው መጪውን ወረርሽኝ ስሙን “በሽታ ኤክስ” በሚል ይዘውታል።

የዓለም ጤና ድርጅት ከእንግዲህ መዘናጋት የለም፤ ለመጪው ወረርሽኝ መዘጋጀት የግድ ነው ሲል ነበር። ምክንያቱም የኮቪድ ወረርሽኝ ዓለም ላይ ያደረሰው ጠባሳ፣ አገራትን ያስከፈለው ዋጋ ቀደም ብሎ ዝግጅት ተደርጎ ቢሆን ኖሮ ይህን ያህል አይሆንም ነበር ይላሉ፤ ጤናውን ዘርፍ የሚመሩት።

ዓለምን ትሪሊዮን ዶላር ያከሰራት ኮቪድ የሕክምና መሠረተ ልማት ከፍተኛ ደረጃ ደርሶ ቢሆን ኖሮ ውድመቱ ቢያንስ ይቀንስ ነበር ይላል የዓለም ጤና ድርጅት።

“በሽታ-ኤክስ’’ ምንድነው?
በሽታው በአሁኑ ወቅት በዓለም ላይ ያለ ሳይሆን፣ ወደፊት ሊመጣ ይችላል ተብሎ የሚገመት ወረርሽኝ ነው።

ለዚያም ነው ቁርጥ ያለ ስም ሳይሰጠው በ”ኤክስ” ፊደል የተወከለው።

ምናልባት ከአንድ አገር ተነስቶ አህጉር የሚያቋርጥ ሊሆን ይችላል። ወይም ደግሞ በተወሰኑ አገራት ጉዳት አድርሶ ሥርጭቱ ሊገታ የሚችል ይሆናል።

ይህ ስያሜ ኮቪድ ከመምጣቱ በፊትም ነበር፣ ለነገሩ። የዓለም ጤና ድርጅት በአውሮፓውያኑ ጥር 2018 ስሙን ጠቀስ አድርጎት ነበር።

በዓለም ጤና ድርጅት አንድ ክፍል አለ። ዋና ሥራው አዳዲስ ሊከሰቱ ለሚችሉ ወረርሽኞች ቀድሞ መዘጋጀት እና መሠረተ ልማቱን ማሰናዳት ነው።

በወረርሽኝ ጊዜ ደግሞ ጥናትና ምርምር ማድረግ እና ለበሽታው የክትባት ሙከራዎችን ማድረግ፣ መድኃኒት መዘጋጀትን ይጨምራል። ዋንኛ ሥራው አንዳች ወረርሽኝ ሲከሰት ቶሎ ሥርጭቱን መግታት ነው።

በቅርብ ዘመን ብዙ ወረርሽኞች ተቀስቅሰው ተመልክተናል።

ሳርስ፣ ስዋይን ጉንፋን፣ ኢቦላ እና ኮቪድ የቅርብ ጊዜ ትዝታዎች ናቸው።

የሚቀጥለው ዓለም አቀፍ ወረርሽን ምን ይሆን የሚለው ባለሙያዎችን ከወዲሁ ማወያየት የጀመረውም ለዚሁ ነው። የጤና ባለሙያዎች መጪው ወረርሽኝ ምናልባት ከእስከዛሬው ሁሉ የከፋ ይሆናል የሚል ፍርሃት አላቸው።


“በሽታ ኤክስ’’ ወረርሽኝ ይሆናል?
ባለፈው ሳምንት በነበረው የዓለም የምጣኔ ሃብት ጉባኤ ላይ አንድ ውይይት ተደርጎ ነበር።

ውይይቱን የመሩት የዓለም ጤና ድርጅት ዳይሬክተር ዶ/ር ቴድሮስ አድሃኖም ገብረየሱስ ነበሩ።

የውይይቱ ርዕስ “ለበሽታ ኤክስ መሰናዳት” ይሰኛል።

ውይይቱ ያተኮረው ወደፊት ከኮቪድ የከፋ ወረርሽኝ ቢከሰት የጤና መዋቅሩ በቶሎ እንዲመክት እንዴት መዘጋጀት ይቻላል የሚል ነበር።

ዶ/ር ቴድሮስ በውይይቱ ላይ ሲናገሩ “አንዳንድ ሰዎች በሽታው ገና ሳይከሰት ለገዳይ ወረርሽኝ መዘጋጀት ፍርሃትን ሊፈጥርባቸው ይችላል። ነገር ግን የሚያዋጣን በቂ ዝግጅት ማድረግ ነው፤ በሰው ልጅ ታሪክ ብዙ ወረርሽኞች ተከስተዋል፤ ወደፊትም ይከሰታሉ” ብለዋል።

የኮቪድ ወረርሽኝ ብዙዎች የጠበቁት አልነበረም። ነገር ግን የወረርሽኝ ባለሙያዎች ይህ ነገር ቀድሞ ሊከሰት እንደሚችል ሲያስጠነቅቁ ነው የኖሩት።

ከእነዚህ ባለሙያዎች መካከል ብዙዎቹ አሁንም ቀድሞ ከእንሰሳት የሚመነጭ ወረርሽኝ ያሰጋቸዋል።

እስከ ዛሬ ከተከሰቱ ወረርሽኞች 75 በመቶዎቹ የሚሆኑት እንሰሳት ወለድ (zoonotic) ናቸው። በሌላ አነጋገር ከእንሰሳት ወደ ሰው የተላለፉ ናቸው።

ኮቪድም እንደዚያ ነው። ቻይና ዉሃን ከሚገኝ የእንሰሳት እርድ ገበያ ወደ ሰው እንደተላለፈ ነው የሚገመተው። የሰው ልጆች ባህሪ፣ የአየር ንብረት መለወጥ፣ የሰው ልጆች ከቦታ ቦታ በፍጥነት መንቀሳቀስ፣ በዱር እንሰሳት አካባቢ የሰው ልጆች መገኘት እና ኑሯቸውን ማወክ እንሰሳት ወለድ ወረርሽ እንዲወለድ ሊያደርግ ይችላል ይላሉ ባለሙያዎች።

የከተሞች መስፋፋት፣ የሕዝብ ብዛት በፍጥነት መጨመር፣ የንግድ መቀላጠፍ በተለይ ወረርሽኞች ሲነሱ በቀላሉ ዓለምን እንዲያዳርሱ ምቹ ሁኔታን ፈጥሯል ይላሉ ባለሙያዎች።


ዓለም ለመጪው ወረርሽኝ ምን ያህል ተዘጋጅቷል?
በዳቮስ የዓለም የምጣኔ ሃብት ጉባኤ ላይ ዶ/ር ቴድሮስ መሥሪያ ቤታቸው ለመጪው ወረርሽኝ ከወዲሁ መሰናዶ መጀመሩን አብስረዋል።

ይህ የወረርሽኝ ፈንድ እና የቴክኖሎጂ ልውውጥ ማዕከልን መክፈት ይጨምራል። ሁለቱ ማዕከላት በደቡብ አፍሪካ ነው የተከፈቱት። አንዱ ዕቅድ በዚያው ማዕከል ክትባት እንዲመረት ማስቻል ነው። ሌላው መጪው ዕቅድ ደግሞ በሃብታም እና ድሃ አገራት መሃል የክትባት ሥርጭትን ፍትሃዊ ማድረግ ነው።

በኮቪድ ጊዜ በዚህ ረገድ ብዙ ክፍተቶች እንደነበሩ ይታወሳል።

የአውሮፓ ኅብረት የበሽታ ቁጥጥር እና መከላከል ባለሥልጣናት በአውሮፓውያኑ 2022 ባወጣው አንድ ሪፖርት፣ አዲስ የጤና መሠረተ ልማት ለመዘርጋት ከመሞከር ይልቅ ያለውን ማጠናከር ወረርሽኝን ቶሎ ለመግታት ይረዳል ብሎ ነበር።

የሚዘረጉ መሠረተ ልማቶች እና የጤና ሥርዓቶች ወረርሽኝ ከመምጣቱ በፊት ሊፈተሹ ይገባልም ብሏል።

በ2022 የዓለም ጤና ድርጅት 10 ምክረ ሐሳቦችን አሳትሞ ነበር። እነዚህ ምክረ ሐሳቦች በተፋጠነ ዘዴ የጤና መረጃን እንዴት መሰብሰብ እና መጋራት ይቻላል፤ እንዴትስ መተንተን ይቻላል የሚለውን ጉዳይ የሚያብራራ ነው።

“የተሳለጠ የበሽታ ቅኝትና ክትትል ዘዴ (disease surveillance system) ወረርሽኝ ሲከሰት ቶሎ እንዲደረስበት ያስችላል። ወረርሽን ከተዛመተ በኋላ የብዙዎን ሕይወት ያሳጣል፤ ብዙ ዋጋ ያስከፍላል፤ መቆጣጠሩም ከአቅም በላይ ይሆናል” ይላል ከምክረ ሐሳቦች አንዱ።

የዓለም ጤና ድርጅት ወረርሽኞች ሁልጊዜም ከሚታወቅ ምንጭ ይነሳሉ ብሎ አያምንም። አንዳንድ ጊዜ ለሰው ልጅ ግልጽ ባልሆኑ እና መኖራቸው እንኳ በማይታወቁ በሽታ አምጪ ተህዋስ (pathogen) ሊነሳ ይችላል።

ለዚህ መፍትሄው ብዙ ሰዎች የክትባት ቴክኖሎጂን ማሳደግ እና ማዘመን ነው ዋናው ብለው ያምናሉ።

በርካታ የኮቪድ ክትባቶች የተመረቱት ወረርሽኙ በተከሰተ በዓመቱ ነው። ይህ በሕክምና ታሪክ አስደናቂ የሚባል ነው።

ወደፊት ሳይንቲስቶች ጊዜን ለመቆጠብ ከዚህ ቀደም ከተመረቱ ክትባቶች በአጭር ጊዜ አቀነባብሮ አዲስ ክትባት የሚፈጠርበትን መንገድ ሊያስቡ ይገባል። ልክ ምግብን ለማሰናዳት መሠረታዊ የሚባሉ አስቤዛዎች እንደሚያስፈልጉ ሁሉ ማለት ነው።

ይህ ሲሆን ነው በአጭር ጊዜ የወረርሽ ሥርጭትን መግታት የሚቻለው፤ አይበለውና ‘በሽታ ኤክስ’ ወረርሽኝ ከተፈጠረ። አለዚያ ‘ወረርሽን ኤክስ’ በሚያደርሰው ጥፋት ኮቪድን የሚያስንቅ ሊሆን ይችላል።
         '''

title  = '''
የዓለም የምጣኔ ሃብት ጉባኤ ላይ ዶር ቴድሮስ መሥሪያ ቤታቸው ለመጪው ወረርሽኝ ከወዲሁ መሰናዶ መጀመሩን አብስረዋል። ይህ የወረርሽኝ ፈንድ እና የቴክኖሎጂ ልውውጥ ማዕከልን መክፈት ይጨምራል። ሌላው መጪው ዕቅድ ደግሞ በሃብታም እና ድሃ አገራት መሃል የክትባት ሥርጭትን ፍትሃዊ ማድረግ ነው። 
የአውሮፓ ኅብረት የበሽታ ቁጥጥር እና መከላከል ባለሥልጣናት በአውሮፓውያኑ ባወጣው አንድ ሪፖርት አዲስ የጤና መሠረተ ልማት ለመዘርጋት ከመሞከር ይልቅ ያለውን ማጠናከር ወረርሽኝን ቶሎ ለመግታት ይረዳል ብሎ ነበር። 
በ የዓለም ጤና ድርጅት ወረርሽኞች ሁልጊዜም ከሚታወቅ ምንጭ ይነሳሉ ብሎ አያምንም። አንዳንድ ጊዜ ለሰው ልጅ ግልጽ ባልሆኑ እና መኖራቸው እንኳ በማይታወቁ በሽታ አምጪ ተህዋስ ሊነሳ ይችላል። ለዚህ መፍትሄው ብዙ ሰዎች የክትባት ቴክኖሎጂን ማሳደግ እና ማዘመን ነው ዋናው ብለው ያምናሉ።
'''
tparser = ExtractiveModel(content)
article_summary = _get_summary(tparser)
rouge_score(title, article_summary)
print("\n\n")
print("Original Summary")
print("\n")
print(title)
print("===================")
print('\n')
print("Generated Summary")
print("\n")
print(article_summary)


ROUGE-1: 0.8051
ROUGE-2: 0.7537
ROUGE-L: 0.6214



Original Summary



የዓለም የምጣኔ ሃብት ጉባኤ ላይ ዶር ቴድሮስ መሥሪያ ቤታቸው ለመጪው ወረርሽኝ ከወዲሁ መሰናዶ መጀመሩን አብስረዋል። ይህ የወረርሽኝ ፈንድ እና የቴክኖሎጂ ልውውጥ ማዕከልን መክፈት ይጨምራል። ሌላው መጪው ዕቅድ ደግሞ በሃብታም እና ድሃ አገራት መሃል የክትባት ሥርጭትን ፍትሃዊ ማድረግ ነው። 
የአውሮፓ ኅብረት የበሽታ ቁጥጥር እና መከላከል ባለሥልጣናት በአውሮፓውያኑ ባወጣው አንድ ሪፖርት አዲስ የጤና መሠረተ ልማት ለመዘርጋት ከመሞከር ይልቅ ያለውን ማጠናከር ወረርሽኝን ቶሎ ለመግታት ይረዳል ብሎ ነበር። 
በ የዓለም ጤና ድርጅት ወረርሽኞች ሁልጊዜም ከሚታወቅ ምንጭ ይነሳሉ ብሎ አያምንም። አንዳንድ ጊዜ ለሰው ልጅ ግልጽ ባልሆኑ እና መኖራቸው እንኳ በማይታወቁ በሽታ አምጪ ተህዋስ ሊነሳ ይችላል። ለዚህ መፍትሄው ብዙ ሰዎች የክትባት ቴክኖሎጂን ማሳደግ እና ማዘመን ነው ዋናው ብለው ያምናሉ።



Generated Summary


የዓለም የምጣኔ ሃብት ጉባኤ ላይ ዶር ቴድሮስ መሥሪያ ቤታቸው ለመጪው ወረርሽኝ ከወዲሁ መሰናዶ መጀመሩን አብስረዋል። ይህ የወረርሽኝ ፈንድ እና የቴክኖሎጂ ልውውጥ ማዕከልን መክፈት ይጨምራል። ሌላው መጪው ዕቅድ ደግሞ በሃብታም እና ድሃ አገራት መሃል የክትባት ሥርጭትን ፍትሃዊ ማድረግ ነው። የአውሮፓ ኅብረት የበሽታ ቁጥጥር እና መከላከል ባለሥልጣናት በአውሮፓውያኑ ባወጣው አንድ ሪፖርት አዲስ የጤና መሠረተ ልማት ለመዘርጋት ከመሞከር ይልቅ ያለውን ማጠናከር ወረርሽኝን ቶሎ ለመግታት ይረዳል ብሎ ነበር። የሚዘረጉ መሠረተ ልማቶች እና የጤና ሥርዓቶች ወረርሽኝ ከመምጣቱ በፊት ሊፈተሹ ይገባልም ብሏል። በ የዓለም ጤና ድርጅት ወረርሽኞች ሁል